### Notebook for the contextualisation of cell-cell communications on cardiomyocytes data using `Tensor2Cell`

- **Developed by**: Carlos Talavera-López
- **Würzburg Institute for Systems Immunology, Faculty of Medicine, Julius-Maximilian-Universität Würzburg**
- **Created**: 240201
- **Latest version**: 240201

### Import required modules

In [3]:
import anndata
import warnings
import numpy as np
import liana as li
import pandas as pd
import scanpy as sc
import plotnine as p9
import decoupler as dc
import cell2cell as c2c
from collections import defaultdict
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean

### Set up working environment

In [ ]:
warnings.filterwarnings('ignore')

sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

### Read in data

In [ ]:
adata = sc.read_h5ad('../../../data/heart_mm_nuclei-23-0092_CMC_states_ctl240131.raw.h5ad') 
adata

In [ ]:
adata.obs["cell_type"] = adata.obs["C_scANVI"].copy()
adata.obs["cell_type"].cat.categories

In [ ]:
sample_key = 'sample'
condition_key = 'genotype'
groupby = 'cell_type'

### Ligand-Receptor Inference by Sample

In [ ]:
li.mt.rank_aggregate.by_sample(
    adata,
    groupby = groupby,
    sample_key = sample_key,
    use_raw = False,
    verbose = True, 
    n_perms = 100, 
    return_all_lrs = True, 
    resource_name = 'mouseconsensus'
    )

In [ ]:
adata.uns["liana_res"].sort_values("magnitude_rank").head(10)

### Building a Tensor

In [ ]:
tensor = li.multi.to_tensor_c2c(adata,
                                sample_key = sample_key,
                                score_key = 'magnitude_rank', # can be any score from liana
                                how = 'outer_cells' # how to join the samples
                                )

In [ ]:
tensor.tensor.shape

In [ ]:
c2c.io.export_variable_with_pickle(tensor, "tensor_tutorial.pkl")

In [ ]:
context_dict = adata.obs[[sample_key, condition_key]].drop_duplicates()
context_dict = dict(zip(context_dict[sample_key], context_dict[condition_key]))
context_dict = defaultdict(lambda: 'Unknown', context_dict)

tensor_meta = c2c.tensor.generate_tensor_metadata(interaction_tensor = tensor,
                                                  metadata_dicts = [context_dict, None, None, None],
                                                  fill_with_order_elements = True
                                                  )

### Running `Tensor-cell2cell`

In [ ]:
tensor = c2c.analysis.run_tensor_cell2cell_pipeline(tensor,
                                                    tensor_meta,
                                                    copy_tensor = True, # Whether to output a new tensor or modifying the original
                                                    rank= None, # Number of factors to perform the factorization. If None, it is automatically determined by an elbow analysis. Here, it was precomuputed.
                                                    tf_optimization = 'regular', # To define how robust we want the analysis to be.
                                                    random_state = 1712, # Random seed for reproducibility
                                                    device = 'cuda', # Device to use. If using GPU and PyTorch, use 'cuda'. For CPU use 'cpu'
                                                    elbow_metric = 'error', # Metric to use in the elbow analysis.
                                                    smooth_elbow = False, # Whether smoothing the metric of the elbow analysis.
                                                    upper_rank = 20, # Max number of factors to try in the elbow analysis
                                                    tf_init = 'random', # Initialization method of the tensor factorization
                                                    tf_svd = 'numpy_svd', # Type of SVD to use if the initialization is 'svd'
                                                    cmaps = None, # Color palettes to use in color each of the dimensions. Must be a list of palettes.
                                                    sample_col = 'Element', # Columns containing the elements in the tensor metadata
                                                    group_col = 'Category', # Columns containing the major groups in the tensor metadata
                                                    output_fig = False, 
                                                    )

In [ ]:
factors, axes = c2c.plotting.tensor_factors_plot(interaction_tensor = tensor,
                                                 metadata = tensor_meta, # This is the metadata for each dimension
                                                 sample_col='Element',
                                                 group_col='Category',
                                                 meta_cmaps = ['viridis', 'Dark2_r', 'tab20', 'tab20'],
                                                 fontsize = 10, # Font size of the figures generated
                                                 )

### Factorization Results

In [ ]:
factors = tensor.factors
factors.keys()

In [ ]:
factors['Contexts']

In [ ]:
lr_loadings = factors['Ligand-Receptor Pairs']
lr_loadings.sort_values("Factor 9", ascending = False).head(10)

### Enrichment analysis of the results using `decoupler` 

In [ ]:
net = dc.get_progeny(organism = 'mouse', top = 5000)

In [ ]:
lr_pairs = li.resource.select_resource('mouseconsensus')
lr_progeny = li.rs.generate_lr_geneset(lr_pairs, net, lr_sep = "^")
lr_progeny.head()

In [ ]:
estimate, pvals = dc.run_mlm(lr_loadings.transpose(), 
                             lr_progeny, 
                             source = "source", 
                             target = "interaction", 
                             use_raw = False)

In [ ]:
dc.plot_barplot(estimate, 
                'Factor 5', 
                vertical = True, 
                cmap = 'coolwarm', 
                vmin = -7, 
                vmax = 7)